In [1]:
import pandas as pd
import re

# Alternative approach that creates a more tabular structure
def restructure_csv_tabular(input_file, output_file):
    # Read the CSV file
    df = pd.read_csv(input_file)
    
    # Get the column names
    columns = df.columns.tolist()
    
    # Initialize list to store the transformed data rows
    transformed_rows = []
    
    # Variables to keep track of current county, state
    current_county = None
    current_state = None
    current_household_type = None
    
    # Process each row
    for i, row in df.iterrows():
        first_column_value = str(row[columns[0]]).strip()
        
        # Check if this row contains county information
        county_match = re.match(r'(.+) County, (.+)', first_column_value)
        if county_match:
            current_county = county_match.group(1)
            current_state = county_match.group(2)
            continue
            
        # Check if this row contains household type information
        if first_column_value in ['Households', 'Families', 'Married-couple families', 'Nonfamily households']:
            current_household_type = first_column_value
            continue
            
        # If this row is an "Estimate" row, extract the data
        if first_column_value == 'Estimate':
            # Create a new row with all data for this household type
            new_row = {
                'County': current_county,
                'State': current_state,
                'Household_Type': current_household_type
            }
            
            # Add all the income data columns
            for j in range(1, len(columns)):
                # Clean up column names for better readability
                clean_col_name = columns[j].replace('Total!!', '').strip()
                new_row[clean_col_name] = row[columns[j]]
                
            transformed_rows.append(new_row)
    
    # Create a DataFrame from the transformed rows
    result_df = pd.DataFrame(transformed_rows)
    
    # Save to CSV
    # result_df.to_csv(output_file, index=False)
    
    return result_df

# Example usage:
# restructure_csv("input_data.csv", "restructured_data.csv")  # For long format
result_df = restructure_csv_tabular("../raw_data/income_2020.csv", "restructured_data_tabular.csv")  # For wide format

In [2]:
result_df.head(10)

,County,State,Household_Type,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income,Mean income,PERCENT ALLOCATED,PERCENT ALLOCATED!!Household income in the past 12 months,PERCENT ALLOCATED!!Family income in the past 12 months,PERCENT ALLOCATED!!Nonfamily income in the past 12 months
0,Autauga,Alabama,Households,"21,559",6.2%,4.6%,12.3%,8.5%,12.7%,17.0%,13.4%,16.4%,4.8%,4.2%,"57,982","75,614",NaN,32.5%,(X),(X)
1,Autauga,Alabama,Families,"15,103",5.2%,2.6%,8.1%,7.2%,11.9%,17.7%,16.0%,19.4%,6.3%,5.7%,"70,730","88,331",NaN,(X),30.1%,(X)
2,Autauga,Alabama,Married-couple families,"11,281",2.3%,1.1%,5.0%,7.3%,8.6%,17.8%,17.4%,25.2%,8.3%,7.1%,"87,738",N,NaN,(X),(X),(X)
3,Autauga,Alabama,Nonfamily households,"6,456",10.3%,9.7%,22.2%,12.3%,13.9%,15.3%,6.5%,8.0%,1.4%,0.4%,"30,041","43,328",NaN,(X),(X),36.1%
4,Baldwin,Alabama,Households,"84,047",5.2%,4.8%,7.7%,10.0%,14.0%,16.8%,13.7%,15.5%,5.8%,6.6%,"61,756","83,626",NaN,34.6%,(X),(X)
5,Baldwin,Alabama,Families,"56,092",2.1%,1.9%,4.6%,7.5%,12.5%,17.7%,16.1%,20.2%,8.2%,9.2%,"79,907","101,693",NaN,(X),37.2%,(X)
6,Baldwin,Alabama,Married-couple families,"46,277",1.0%,0.8%,3.1%,5.9%,11.4%,17.0%,17.9%,22.9%,9.1%,10.9%,"88,867","111,411",NaN,(X),(X),(X)
7,Baldwin,Alabama,Nonfamily households,"27,955",11.8%,10.5%,14.3%,15.9%,16.5%,15.6%,7.4%,5.6%,1.0%,1.4%,"32,861","45,788",NaN,(X),(X),28.4%
8,Barbour,Alabama,Households,"9,322",14.6%,7.6%,18.4%,9.4%,12.5%,16.3%,7.3%,9.1%,2.2%,2.6%,"34,990","51,557",NaN,43.9%,(X),(X)
9,Barbour,Alabama,Families,"6,083",11.1%,5.4%,12.2%,12.0%,12.2%,19.5%,10.2%,10.7%,3.3%,3.3%,"45,836","60,687",NaN,(X),47.4%,(X)


In [3]:
# Filter result_df to keep only rows where Household_Type is "Households"
filtered_df = result_df[result_df['Household_Type'] == 'Households']

# Reset the index to have consecutive integers (optional)
filtered_df = filtered_df.reset_index(drop=True)

# Display the first few rows to verify the result
filtered_df.head()

,County,State,Household_Type,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income,Mean income,PERCENT ALLOCATED,PERCENT ALLOCATED!!Household income in the past 12 months,PERCENT ALLOCATED!!Family income in the past 12 months,PERCENT ALLOCATED!!Nonfamily income in the past 12 months
0,Autauga,Alabama,Households,"21,559",6.2%,4.6%,12.3%,8.5%,12.7%,17.0%,13.4%,16.4%,4.8%,4.2%,"57,982","75,614",NaN,32.5%,(X),(X)
1,Baldwin,Alabama,Households,"84,047",5.2%,4.8%,7.7%,10.0%,14.0%,16.8%,13.7%,15.5%,5.8%,6.6%,"61,756","83,626",NaN,34.6%,(X),(X)
2,Barbour,Alabama,Households,"9,322",14.6%,7.6%,18.4%,9.4%,12.5%,16.3%,7.3%,9.1%,2.2%,2.6%,"34,990","51,557",NaN,43.9%,(X),(X)
3,Bibb,Alabama,Households,"7,259",11.0%,7.1%,10.5%,9.1%,11.2%,19.4%,15.6%,11.2%,2.8%,2.2%,"51,721","61,655",NaN,35.6%,(X),(X)
4,Blount,Alabama,Households,"21,205",10.1%,4.6%,11.0%,11.2%,14.5%,17.9%,10.8%,12.4%,4.9%,2.7%,"48,922","66,360",NaN,38.2%,(X),(X)


In [4]:
filtered_df.shape

(3221, 20)

In [5]:
heart_df = pd.read_csv("../cleaned_data/heart_disease_mortality_cleaned.csv")

In [6]:
heart_df.head(10)

,Year,LocationAbbr,LocationDesc,GeographicLevel,Heart Disease Mortality,Data_Value_Unit,Data_Value_Type,Sex,ethnicity,LocationID,Y_lat,X_lon,Georeference
0,2020,AK,Denali,County,348.8,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Male,White,2068,63.678820,-149.960801,POINT (-149.9608012 63.67881971)
1,2020,CA,California,State,230.1,"per 100,000 population","Age-adjusted, 3-year Average Rate",Male,More than one race,6,37.241400,-119.601000,POINT (-119.601 37.2414)
2,2020,CO,Park County,County,135.9,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,White,8093,39.115616,-105.708698,POINT (-105.7086982 39.11561621)
3,2020,FL,Walton County,County,126.5,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Asian,12131,30.655966,-86.157457,POINT (-86.15745736 30.65596581)
4,2020,GA,Whitfield County,County,155.1,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Hispanic,13313,34.803854,-84.962112,POINT (-84.96211184 34.80385386)
5,2020,IA,Ida County,County,168.0,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Overall,Hispanic,19093,42.387689,-95.523143,POINT (-95.52314316 42.38768939)
6,2020,ID,Teton County,County,191.9,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,White,16081,43.755726,-111.205298,POINT (-111.2052979 43.75572633)
7,2020,KS,Sedgwick County,County,97.8,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Asian,20173,37.680329,-97.454044,POINT (-97.45404365 37.68032871)
8,2020,KY,Shelby County,County,152.3,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Hispanic,21211,38.219932,-85.203751,POINT (-85.20375065 38.21993221)
9,2020,MD,Charles County,County,89.3,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Asian,24017,38.516117,-76.992402,POINT (-76.99240245 38.51611705)


In [7]:
# Code to remove "county" from LocationDesc values
def clean_county_names(heart_df):
    # Create a copy of the dataframe to avoid modifying the original
    df_cleaned = heart_df.copy()
    
    # Function to remove 'county' from a string (case insensitive)
    def remove_county(location):
        if isinstance(location, str):  # Check if the value is a string
            # Remove ' County', 'County ', or ' County ' with case insensitivity
            return location.replace(' County', '').replace('County ', '').replace(' County ', ' ')
        return location  # Return the original value if not a string
    
    # Apply the function to the LocationDesc column
    df_cleaned['LocationDesc'] = df_cleaned['LocationDesc'].apply(remove_county)
    
    return df_cleaned

# Apply the function to your dataframe
heart_df_cleaned = clean_county_names(heart_df)

# Display the first 10 rows to see the results
heart_df_cleaned.head(10)

,Year,LocationAbbr,LocationDesc,GeographicLevel,Heart Disease Mortality,Data_Value_Unit,Data_Value_Type,Sex,ethnicity,LocationID,Y_lat,X_lon,Georeference
0,2020,AK,Denali,County,348.8,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Male,White,2068,63.678820,-149.960801,POINT (-149.9608012 63.67881971)
1,2020,CA,California,State,230.1,"per 100,000 population","Age-adjusted, 3-year Average Rate",Male,More than one race,6,37.241400,-119.601000,POINT (-119.601 37.2414)
2,2020,CO,Park,County,135.9,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,White,8093,39.115616,-105.708698,POINT (-105.7086982 39.11561621)
3,2020,FL,Walton,County,126.5,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Asian,12131,30.655966,-86.157457,POINT (-86.15745736 30.65596581)
4,2020,GA,Whitfield,County,155.1,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Hispanic,13313,34.803854,-84.962112,POINT (-84.96211184 34.80385386)
5,2020,IA,Ida,County,168.0,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Overall,Hispanic,19093,42.387689,-95.523143,POINT (-95.52314316 42.38768939)
6,2020,ID,Teton,County,191.9,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,White,16081,43.755726,-111.205298,POINT (-111.2052979 43.75572633)
7,2020,KS,Sedgwick,County,97.8,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Asian,20173,37.680329,-97.454044,POINT (-97.45404365 37.68032871)
8,2020,KY,Shelby,County,152.3,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Hispanic,21211,38.219932,-85.203751,POINT (-85.20375065 38.21993221)
9,2020,MD,Charles,County,89.3,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Asian,24017,38.516117,-76.992402,POINT (-76.99240245 38.51611705)


In [8]:
heart_df_cleaned.shape

(34430, 13)

In [9]:
# 두 데이터프레임을 County와 LocationDesc 칼럼을 기준으로 병합하기
# filtered_df의 County 칼럼과 heart_df_cleaned의 LocationDesc 칼럼을 기준으로 병합

# 병합(merge) 수행
filtered_df_unique = filtered_df.drop_duplicates(subset=['County'])
merged_df = pd.merge(
    filtered_df_unique,
    heart_df_cleaned,
    left_on='County',
    right_on='LocationDesc',
    how='right'  # inner join: 두 데이터프레임에서 매칭되는 값만 유지
)

# 결과 확인
merged_df.head()

,County,State,Household_Type,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999",...,GeographicLevel,Heart Disease Mortality,Data_Value_Unit,Data_Value_Type,Sex,ethnicity,LocationID,Y_lat,X_lon,Georeference
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,County,348.8,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Male,White,2068,63.678820,-149.960801,POINT (-149.9608012 63.67881971)
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,State,230.1,"per 100,000 population","Age-adjusted, 3-year Average Rate",Male,More than one race,6,37.241400,-119.601000,POINT (-119.601 37.2414)
2,Park,Colorado,Households,"6,987",1.6%,2.9%,10.4%,8.0%,7.0%,18.1%,...,County,135.9,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,White,8093,39.115616,-105.708698,POINT (-105.7086982 39.11561621)
3,Walton,Florida,Households,"28,635",5.9%,3.5%,8.6%,7.9%,12.3%,18.6%,...,County,126.5,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Asian,12131,30.655966,-86.157457,POINT (-86.15745736 30.65596581)
4,Whitfield,Georgia,Households,"36,412",5.0%,4.6%,12.5%,12.3%,15.5%,20.4%,...,County,155.1,"per 100,000 population","Age-adjusted, Spatially Smoothed, 3-year Avera...",Female,Hispanic,13313,34.803854,-84.962112,POINT (-84.96211184 34.80385386)


In [10]:
merged_df.shape

(34430, 33)

In [11]:
merged_df.columns

Index(['County', 'State', 'Household_Type', 'Total', 'Less than $10,000',
       '$10,000 to $14,999', '$15,000 to $24,999', '$25,000 to $34,999',
       '$35,000 to $49,999', '$50,000 to $74,999', '$75,000 to $99,999',
       '$100,000 to $149,999', '$150,000 to $199,999', '$200,000 or more',
       'Median income', 'Mean income', 'PERCENT ALLOCATED',
       'PERCENT ALLOCATED!!Household income in the past 12 months',
       'PERCENT ALLOCATED!!Family income in the past 12 months',
       'PERCENT ALLOCATED!!Nonfamily income in the past 12 months', 'Year',
       'LocationAbbr', 'LocationDesc', 'GeographicLevel',
       'Heart Disease Mortality', 'Data_Value_Unit', 'Data_Value_Type', 'Sex',
       'ethnicity', 'LocationID', 'Y_lat', 'X_lon', 'Georeference'],
      dtype='object')

In [ ]:
# 지정된 칼럼들 제거하기
columns_to_drop = [
    'Household_Type',
    'PERCENT ALLOCATED!!Household income in the past 12 months', 
    'PERCENT ALLOCATED!!Family income in the past 12 months', 
    'PERCENT ALLOCATED!!Nonfamily income in the past 12 months', 
    'Year',
    'Data_Value_Unit', 
    'Data_Value_Type',
    'PERCENT ALLOCATED',
    'LocationAbbr',
    'Y_lat', 
    'X_lon', 
    'Georeference',
    'LocationID',
    'LocationDesc', 
    'GeographicLevel'
]

# drop 메서드를 사용하여 칼럼 제거
merged_df = merged_df.drop(columns=columns_to_drop)

# 결과 확인
print(merged_df.head())

      County     State   Total Less than $10,000 $10,000 to $14,999  \
0        NaN       NaN     NaN               NaN                NaN   
1        NaN       NaN     NaN               NaN                NaN   
2       Park  Colorado   6,987              1.6%               2.9%   
3     Walton   Florida  28,635              5.9%               3.5%   
4  Whitfield   Georgia  36,412              5.0%               4.6%   

  $15,000 to $24,999 $25,000 to $34,999 $35,000 to $49,999 $50,000 to $74,999  \
0                NaN                NaN                NaN                NaN   
1                NaN                NaN                NaN                NaN   
2              10.4%               8.0%               7.0%              18.1%   
3               8.6%               7.9%              12.3%              18.6%   
4              12.5%              12.3%              15.5%              20.4%   

  $75,000 to $99,999 $100,000 to $149,999 $150,000 to $199,999  \
0                NaN

In [13]:
merged_df = merged_df.dropna()

In [14]:
merged_df.head(10)

,County,State,Total,"Less than $10,000","$10,000 to $14,999","$15,000 to $24,999","$25,000 to $34,999","$35,000 to $49,999","$50,000 to $74,999","$75,000 to $99,999","$100,000 to $149,999","$150,000 to $199,999","$200,000 or more",Median income,Mean income,LocationDesc,GeographicLevel,Heart Disease Mortality,Sex,ethnicity
2,Park,Colorado,"6,987",1.6%,2.9%,10.4%,8.0%,7.0%,18.1%,15.5%,21.1%,10.5%,4.8%,"76,611","91,905",Park,County,135.9,Female,White
3,Walton,Florida,"28,635",5.9%,3.5%,8.6%,7.9%,12.3%,18.6%,13.8%,17.1%,5.6%,6.7%,"67,390","86,296",Walton,County,126.5,Female,Asian
4,Whitfield,Georgia,"36,412",5.0%,4.6%,12.5%,12.3%,15.5%,20.4%,11.5%,11.9%,2.6%,3.7%,"50,055","67,316",Whitfield,County,155.1,Female,Hispanic
5,Ida,Iowa,"2,972",7.4%,4.1%,9.1%,9.6%,16.6%,16.5%,16.4%,12.4%,4.5%,3.4%,"54,219","72,389",Ida,County,168.0,Overall,Hispanic
6,Teton,Idaho,"4,290",2.1%,4.6%,7.2%,7.2%,8.5%,21.7%,9.3%,21.1%,15.3%,2.9%,"73,274","93,860",Teton,County,191.9,Female,White
7,Sedgwick,Colorado,954,6.6%,7.7%,17.2%,10.0%,12.4%,15.9%,10.9%,13.5%,2.0%,3.9%,"43,875","66,732",Sedgwick,County,97.8,Female,Asian
8,Shelby,Alabama,"80,756",3.8%,3.0%,6.1%,6.2%,10.8%,17.6%,13.9%,20.7%,8.1%,9.8%,"78,889","103,042",Shelby,County,152.3,Female,Hispanic
9,Charles,Maryland,"57,388",3.5%,1.4%,3.5%,4.6%,7.1%,14.1%,13.5%,24.8%,13.6%,13.8%,"103,678","119,477",Charles,County,89.3,Female,Asian
10,Phelps,Missouri,"18,213",10.7%,5.8%,12.5%,9.9%,15.0%,18.1%,10.3%,11.1%,3.7%,2.8%,"44,987","62,126",Phelps,County,374.0,Overall,White
11,Mississippi,Arkansas,"16,623",8.3%,6.3%,16.6%,10.8%,14.3%,17.1%,8.5%,11.6%,3.6%,3.0%,"42,986","61,177",Mississippi,State,167.6,Female,Asian


In [15]:
merged_df.shape

(32111, 20)